In [ ]:
from google.colab import drive
drive.mount('/content/Mydrive')
%cd "/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon"

In [ ]:
!pip install geopandas

import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from datetime import datetime
import numpy as np

area = gpd.GeoDataFrame.from_file('Preprocessing/data/LSMD_ADM_SECT_UMD_11.shp', encoding='cp949') # 경로 수정
grid = gpd.GeoDataFrame.from_file('Seoul_Grid/grid.shp') # (절대)경로 개별적 수정

In [ ]:
area = gpd.GeoDataFrame.from_file('Preprocessing/data/LSMD_ADM_SECT_UMD_11.shp', encoding='cp949') # 경로 수정
grid = gpd.GeoDataFrame.from_file('Seoul_Grid/grid.shp') # (절대)경로 개별적 수정

In [ ]:
df_final = pd.read_pickle('final.pkl')

In [ ]:
df_final

In [ ]:
# S-DoT 유동인구
population = pd.read_csv('Preprocessing/전처리완료/pop_joined.csv', usecols=['사이트명','방문자수','등록일자','la','lo'])
gpd_pop = gpd.GeoDataFrame(population,geometry = gpd.points_from_xy(x = population.lo, y= population.la, crs='WGS84'))
gpd_pop = gpd_pop.to_crs(epsg=5179)

In [ ]:
gpd_pop = gpd_pop.sort_values(by=['사이트명','등록일자'])
gpd_pop.등록일자 = pd.to_datetime(gpd_pop.등록일자)

In [ ]:
gpd_pop = gpd_pop.loc[gpd_pop.등록일자>pd.Timestamp(2021,8,31,23),:]
gpd_pop = gpd_pop.loc[gpd_pop.등록일자<pd.Timestamp(2022,1,1),:]
gpd_pop = gpd_pop.loc[gpd_pop.사이트명 != 0,:]

In [ ]:
sites = gpd_pop.사이트명.unique()

In [ ]:
gpd_pop.columns = ['사이트명', '방문자수', 'dt', 'la', 'lo', 'geometry']

In [ ]:
gpd_pop

In [ ]:
for i in tqdm(range(len(sites))):
    df_site = gpd_pop.loc[gpd_pop.사이트명==sites[i],['방문자수','dt','geometry']].reset_index()
    idx = np.where(grid.contains(df_site.geometry[0]))
    if len(idx[0]) > 0 :
        df_final = pd.merge(df_final, df_site, left_on = ['dt','geometry'], right_on=['dt','geometry'], how = 'left')

df_final.to_pickle('final.pkl')